In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset

# Load litqa dataset
litqa_train = load_dataset("Anshu132002/litqaTrain", split="train")
litqa_test = load_dataset("Anshu132002/litqaTest", split="test")

# Load pubmedqa dataset
pubmedqa_train = load_dataset("Anshu132002/pubmedqaTrain", split="train")
pubmedqa_test = load_dataset("Anshu132002/pubmedqaTest", split="test")


In [ ]:
prompt_litqa = """Below is a question along with a key passage and distractors. Provide only the ideal answer as single word or two to three word only.

### Question:
{}

### Key Passage:
{}

### Distractors:
{}

Answer:"""


In [ ]:
prompt_pubmedqa = """Below is a question along with context. Provide a answer maximum 3-4 lines.

### Question:

{}

### Context:

{}

### Answer:

{}"""


In [ ]:
def preprocess_litqa(examples):
    texts = []
    for question, key_passage, distractors, ideal in zip(
        examples["question"],
        examples["key-passage"],
        examples["distractors"],
        examples["ideal"],
    ):
        text = prompt_litqa.format(question, key_passage, distractors, ideal)
        texts.append(text)
    return {"text": texts}


In [ ]:
def preprocess_pubmedqa(examples):
    texts = []
    for question, context, long_answer in zip(
        examples["QUESTION"],
        examples["CONTEXTS"],
        examples["LONG_ANSWER"],
    ):
        text = prompt_pubmedqa.format(question, context, long_answer)
        texts.append(text)
    return {"text": texts}


In [ ]:
# Preprocess litqa datasets
litqa_train = litqa_train.map(preprocess_litqa, batched=True)
litqa_test = litqa_test.map(preprocess_litqa, batched=True)

# Preprocess pubmedqa datasets
pubmedqa_train = pubmedqa_train.map(preprocess_pubmedqa, batched=True)
pubmedqa_test = pubmedqa_test.map(preprocess_pubmedqa, batched=True)


Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
from datasets import concatenate_datasets

# Combine training datasets
train_dataset = concatenate_datasets([litqa_train, pubmedqa_train])

# Combine test datasets
test_dataset = concatenate_datasets([litqa_test, pubmedqa_test])


In [ ]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

# Set parameters
max_seq_length = 150
dtype = None
load_in_4bit = True

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3.5-mini-instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1,  # Increase for full training
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Start training
trainer.train()


Map (num_proc=2):   0%|          | 0/1095 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,095 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss


TrainOutput(global_step=1, training_loss=2.549076557159424, metrics={'train_runtime': 6.1927, 'train_samples_per_second': 1.292, 'train_steps_per_second': 0.161, 'total_flos': 27017736192000.0, 'train_loss': 2.549076557159424, 'epoch': 0.0072992700729927005})

In [ ]:

from datasets import load_dataset
# Load litqa dataset
litqa_train = load_dataset("Anshu132002/litqaTrain", split="train")
litqa_test = load_dataset("Anshu132002/litqaTest", split="test")

pubmedqa_train = load_dataset("Anshu132002/pubmedqaTrain", split="train")
pubmedqa_test = load_dataset("Anshu132002/pubmedqaTest", split="test")

prompt_litqa = """Answer the following question based on the key passage. Provide only the ideal answer as a single word or a few words.

Question:
{}

Key Passage:
{}

Answer:"""

prompt_pubmedqa = """Answer the following question based on the context. Provide an answer in 3-4 lines.

Question:
{}

Context:
{}

Answer:"""

def preprocess_litqa(examples):
    texts = []
    for question, key_passage in zip(
        examples["question"],
        examples["key-passage"],
    ):
        text = prompt_litqa.format(question, key_passage)
        texts.append(text)
    return {"text": texts}

def preprocess_pubmedqa(examples):
    texts = []
    for question, context in zip(
        examples["QUESTION"],
        examples["CONTEXTS"],
    ):
        text = prompt_pubmedqa.format(question, context)
        texts.append(text)
    return {"text": texts}

litqa_train = litqa_train.map(preprocess_litqa, batched=True)

pubmedqa_train = pubmedqa_train.map(preprocess_pubmedqa, batched=True)

from datasets import concatenate_datasets

train_dataset = concatenate_datasets([litqa_train, pubmedqa_train])


from unsloth import FastLanguageModel
import torch

# Set parameters
max_seq_length = 150
dtype = None
load_in_4bit = True

# Load the model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3.5-mini-instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1, 
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# Start training
trainer.train()

from tqdm import tqdm 

FastLanguageModel.for_inference(model) 

def preprocess_test_dataset(dataset, format_string, fields):
    texts = []
    for entry in zip(*[dataset[field] for field in fields]):
        text = format_string.format(*entry)
        texts.append(text)
    return texts

litqa_test_texts = preprocess_test_dataset(
    litqa_test,
    prompt_litqa,
    ["question", "key-passage"]
)
litqa_test = litqa_test.add_column("text", litqa_test_texts)

pubmedqa_test_texts = preprocess_test_dataset(
    pubmedqa_test,
    prompt_pubmedqa,
    ["QUESTION", "CONTEXTS"]
)
pubmedqa_test = pubmedqa_test.add_column("text", pubmedqa_test_texts)

def generate_predictions_for_all(model, tokenizer, test_dataset, max_seq_length):
    predictions = []
    inputs_list = test_dataset['text']

    for idx, input_text in enumerate(tqdm(inputs_list, desc="Generating Predictions")):
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=max_seq_length,
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                use_cache=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                num_beams=1,
                do_sample=False,
            )

        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        answer = output_text[len(input_text):].strip()

        answer_line = answer.split('\n')[0].strip()
        predictions.append(answer_line)

        print(f"Example {idx + 1}: {predictions[-1]}")

    return predictions

print("\nGenerating predictions for LitQA Test Dataset...")
litqa_predictions = generate_predictions_for_all(model, tokenizer, litqa_test, max_seq_length)

print("\nGenerating predictions for PubMedQA Test Dataset...")
pubmedqa_predictions = generate_predictions_for_all(model, tokenizer, pubmedqa_test, max_seq_length)

import json

results_litqa = []

for idx in range(len(litqa_test)):
    result = {
        "question": litqa_test["question"][idx],
        "key-passage": litqa_test["key-passage"][idx],
        "generated_answer": litqa_predictions[idx]
    }
    results_litqa.append(result)

with open("litqa_predictions.json", "w") as f:
    json.dump(results_litqa, f, indent=4)

results_pubmedqa = []

# for idx in range(len(pubmedqa_test)):
#     result = {
#         "QUESTION": pubmedqa_test["QUESTION"][idx],
#         "CONTEXTS": pubmedqa_test["CONTEXTS"][idx],
#         "generated_answer": pubmedqa_predictions[idx]
#     }
#     results_pubmedqa.append(result)

# with open("pubmedqa_predictions.json", "w") as f:
#     json.dump(results_pubmedqa, f, indent=4)


Map:   0%|          | 0/145 [00:00<?, ? examples/s]

Map:   0%|          | 0/950 [00:00<?, ? examples/s]

==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Map (num_proc=2):   0%|          | 0/1095 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,095 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1
 "-____-"     Number of trainable parameters = 29,884,416


...

Example 48: 


Generating Predictions:  98%|█████████▊| 49/50 [02:55<00:03,  3.47s/it]

Example 49: 


Generating Predictions: 100%|██████████| 50/50 [02:59<00:00,  3.59s/it]

Example 50: 


In [ ]:

for idx in range(len(pubmedqa_test)):
    result = {
        "QUESTION": pubmedqa_test["QUESTION"][idx],
        "CONTEXTS": pubmedqa_test["CONTEXTS"][idx],
        "generated_answer": pubmedqa_predictions[idx]
    }
    results_pubmedqa.append(result)

with open("pubmedqa_predictions.json", "w") as f:
    json.dump(results_pubmedqa, f, indent=4)

In [ ]:

!pip install datasets

from datasets import load_dataset

# Load datasets
litqa_train = load_dataset("Anshu132002/litqaTrain", split="train")
litqa_test = load_dataset("Anshu132002/litqaTest", split="test")

pubmedqa_train = load_dataset("Anshu132002/pubmedqaTrain", split="train")
pubmedqa_test = load_dataset("Anshu132002/pubmedqaTest", split="test")

def messages_to_prompt(messages):
    prompt = ''
    for message in messages:
        role = message['role']
        content = message['content']
        if role == 'system':
            prompt += f"System: {content}\n"
        elif role == 'user':
            prompt += f"User: {content}\n"
        elif role == 'assistant':
            prompt += f"Assistant: {content}\n"
    prompt += 'Assistant: '
    return prompt

def preprocess_litqa(examples):
    texts = []
    for question, key_passage, ideal in zip(
        examples["question"],
        examples["key-passage"],
        examples["ideal"],
    ):
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": f"Question: {question}\nKey Passage: {key_passage}\nProvide a concise answer."},
            {"role": "assistant", "content": f"{ideal}"}
        ]
        text = messages_to_prompt(messages)
        texts.append(text)
    return {"text": texts}

def preprocess_pubmedqa(examples):
    texts = []
    for question, context, long_answer in zip(
        examples["QUESTION"],
        examples["CONTEXTS"],
        examples["LONG_ANSWER"],
    ):
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": f"Question: {question}\nContext: {context}\nProvide a detailed answer."},
            {"role": "assistant", "content": f"{long_answer}"}
        ]
        text = messages_to_prompt(messages)
        texts.append(text)
    return {"text": texts}

litqa_train = litqa_train.map(preprocess_litqa, batched=True)

pubmedqa_train = pubmedqa_train.map(preprocess_pubmedqa, batched=True)

from datasets import concatenate_datasets

train_dataset = concatenate_datasets([litqa_train, pubmedqa_train])

from unsloth import FastLanguageModel
import torch

max_seq_length = 512
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Phi-3.5-mini-instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
        random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=1000,  
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

trainer.train()

from tqdm import tqdm

FastLanguageModel.for_inference(model)

def preprocess_test_dataset(examples, fields):
    texts = []
    for entry in zip(*[examples[field] for field in fields]):
        if 'key-passage' in fields:
            question, key_passage = entry
            messages = [
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": f"Question: {question}\nKey Passage: {key_passage}\nProvide a concise answer."},
            ]
        else:
            question, context = entry
            messages = [
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": f"Question: {question}\nContext: {context}\nProvide a detailed answer."},
            ]
        text = messages_to_prompt(messages)
        texts.append(text)
    return texts

litqa_test_texts = preprocess_test_dataset(
    litqa_test,
    ["question", "key-passage"]
)
litqa_test = litqa_test.add_column("text", litqa_test_texts)

pubmedqa_test_texts = preprocess_test_dataset(
    pubmedqa_test,
    ["QUESTION", "CONTEXTS"]
)
pubmedqa_test = pubmedqa_test.add_column("text", pubmedqa_test_texts)

def generate_predictions_for_all(model, tokenizer, test_dataset):
    predictions = []
    inputs_list = test_dataset['text']

    for idx, input_text in enumerate(tqdm(inputs_list, desc="Generating Predictions")):
        inputs = tokenizer(
            input_text,
            return_tensors="pt",
            truncation=True,
            max_length=max_seq_length,
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                num_beams=1,
                do_sample=False,
            )

        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        assistant_idx = output_text.find('Assistant:')
        if assistant_idx != -1:
            assistant_reply = output_text[assistant_idx + len('Assistant:'):].strip()
            assistant_reply = assistant_reply.split('\n')[0].strip()
            predictions.append(assistant_reply)
        else:
            predictions.append(output_text.strip())

        print(f"Example {idx + 1}: {predictions[-1]}")

    return predictions

print("\nGenerating predictions for LitQA Test Dataset...")
litqa_predictions = generate_predictions_for_all(model, tokenizer, litqa_test)

print("\nGenerating predictions for PubMedQA Test Dataset...")
pubmedqa_predictions = generate_predictions_for_all(model, tokenizer, pubmedqa_test)

import json

results_litqa = []

for idx in range(len(litqa_test)):
    result = {
        "question": litqa_test["question"][idx],
        "key-passage": litqa_test["key-passage"][idx],
        "generated_answer": litqa_predictions[idx]
    }
    results_litqa.append(result)

with open("litqa_predictions.json", "w") as f:
    json.dump(results_litqa, f, indent=4)

results_pubmedqa = []

for idx in range(len(pubmedqa_test)):
    result = {
        "QUESTION": pubmedqa_test["QUESTION"][idx],
        "CONTEXTS": pubmedqa_test["CONTEXTS"][idx],
        "generated_answer": pubmedqa_predictions[idx]
    }
    results_pubmedqa.append(result)

with open("pubmedqa_predictions.json", "w") as f:
    json.dump(results_pubmedqa, f, indent=4)


==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,095 | Num Epochs = 8
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
10,1.798400
20,1.538800
30,1.432800
40,1.441800
50,1.436900
60,1.421100
70,1.463700
80,1.391500
90,1.457500
100,1.452900



Generating predictions for LitQA Test Dataset...


Generating Predictions:   2%|▏         | 1/49 [00:10<08:12, 10.27s/it]

...

Example 48: ​​AUC0-24 and Cmin show a direct correlation with vancomycin efficacy and safety, respectively. AUC0-24 may be a more representative index than peak concentration for TDM of many drugs with a linear pharmacokinetic profile.


Generating Predictions:  98%|█████████▊| 49/50 [08:30<00:10, 10.44s/it]

Example 49: 2-year follow-up revealed that PRP gel can reduce postoperative pain but has no effect on swelling, interincisal opening, and socket healing.


Generating Predictions: 100%|██████████| 50/50 [08:41<00:00, 10.42s/it]

Example 50: 57% of paediatric medicines have an endogenous pH<5.5, and the majority are SF formulations. This is probably because SF formulations are more prone to acidic constituents, such as preservatives and flavouring agents, being incorporated into the formulation.
